In [ ]:
import argparse, glob, os
import numpy as np
import cv2

In [41]:

IMG_DIR="calibration"
OUT="calibration_output/calib_aruco.npz"
os.makedirs(os.path.dirname(OUT),exist_ok=True)

tag=0.06
d=cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
p=cv2.aruco.DetectorParameters()

if hasattr(cv2.aruco,"ArucoDetector"):
    det=cv2.aruco.ArucoDetector(d,p)
    def detect(im): return det.detectMarkers(im)
else:
    def detect(im): return cv2.aruco.detectMarkers(im,d,parameters=p)

paths=sorted(glob.glob(os.path.join(IMG_DIR,"*.jpeg")))


obj=np.array([[0,0,0],[tag,0,0],[tag,tag,0],[0,tag,0]],np.float32)
objl=[];imgl=[]
sz=None;n=0

for pth in paths:
    im=cv2.imread(pth)
    if im is None: continue
    if sz is None: sz=(im.shape[1],im.shape[0])
    c,i,_=detect(im)
    if i is None or len(c)==0: continue
    u=c[0].reshape(4,2).astype(np.float32)
    imgl.append(u); objl.append(obj.copy()); n+=1

r,K,dist,_,_=cv2.calibrateCamera(objl, imgl, sz, None, None)
print("rms",float(r))
print("K\n",K)
print("dist",dist.ravel())
np.savez(OUT, K=K, dist=dist, rms=r, img_size=sz)


rms 4.289607529882952
K
 [[4.21492395e+03 0.00000000e+00 3.17479184e+03]
 [0.00000000e+00 3.97244329e+03 1.55372480e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist [-0.44676877  1.47553326  0.00669606  0.05712767 -1.51373905]


In [42]:
paths = sorted(glob.glob(os.path.join("calibration","*.jpeg")))
os.makedirs("calibration_output", exist_ok=True)

for pth in paths:
    im = cv2.imread(pth)
    if im is None: 
        continue
    c,i,_ = detect(im)
    vis = im.copy()

    if i is not None and len(c)>0:
        for cc in c:
            pts = cc.reshape(-1,2).astype(int)
            for j in range(4):
                p1 = tuple(pts[j])
                p2 = tuple(pts[(j+1)%4])
                cv2.line(vis, p1, p2, (0,255,0), 4)
                cv2.circle(vis, p1, 8, (0,0,255), -1)

    cv2.imwrite(os.path.join("calibration_output", "vis_"+os.path.basename(pth)), vis)

In [45]:
tag_size = 0.06
obj = np.array([
    [0, 0, 0],
    [tag_size, 0, 0],
    [tag_size, tag_size, 0],
    [0, tag_size, 0]
], np.float32)

paths = sorted(glob.glob(os.path.join("airpods", "*.jpeg")))

c2w_list = []
img_names = []
imgs = []

for pth in paths:
    img = cv2.imread(pth)
    if img is None:
        continue

    corners, ids, _ = detect(img)
    if ids is None or len(corners) == 0:
        continue

    uv = corners[0].reshape(4, 2).astype(np.float32)

    success, rvec, tvec = cv2.solvePnP(obj, uv, K, dist)
    if not success:
        continue

    R, _ = cv2.Rodrigues(rvec)
    Rt = R.T
    t = tvec.reshape(3, 1)

    c2w = np.hstack([Rt, -Rt @ t])

    c2w_list.append(c2w)
    img_names.append(os.path.basename(pth))
    imgs.append(img)
print(img_names)
np.savez("calibration_output/poses_pnp.npz",
         c2ws=np.array(c2w_list, dtype=np.float32),
         names=np.array(img_names),
         K=K,
         dist=dist)



['IMG_1390.jpeg', 'IMG_1391.jpeg', 'IMG_1392.jpeg', 'IMG_1394.jpeg', 'IMG_1395.jpeg', 'IMG_1397.jpeg', 'IMG_1398.jpeg', 'IMG_1399.jpeg', 'IMG_1401.jpeg', 'IMG_1402.jpeg', 'IMG_1406.jpeg', 'IMG_1407.jpeg', 'IMG_1409.jpeg', 'IMG_1410.jpeg', 'IMG_1411.jpeg', 'IMG_1413.jpeg', 'IMG_1417.jpeg', 'IMG_1419.jpeg', 'IMG_1420.jpeg', 'IMG_1421.jpeg', 'IMG_1427.jpeg', 'IMG_1429.jpeg', 'IMG_1432.jpeg', 'IMG_1434.jpeg', 'IMG_1436.jpeg', 'IMG_1437.jpeg', 'IMG_1439.jpeg', 'IMG_1440.jpeg', 'IMG_1442.jpeg', 'IMG_1444.jpeg', 'IMG_1446.jpeg', 'IMG_1447.jpeg', 'IMG_1449.jpeg', 'IMG_1450.jpeg', 'IMG_1451.jpeg', 'IMG_1452.jpeg', 'IMG_1453.jpeg', 'IMG_1454.jpeg', 'IMG_1456.jpeg', 'IMG_1457.jpeg', 'IMG_1459.jpeg', 'IMG_1460.jpeg', 'IMG_1461.jpeg', 'IMG_1462.jpeg', 'IMG_1463.jpeg', 'IMG_1464.jpeg', 'IMG_1465.jpeg', 'IMG_1466.jpeg', 'IMG_1467.jpeg', 'IMG_1468.jpeg']


In [46]:
import viser
import time

poses = np.load("calibration_output/poses_pnp.npz", allow_pickle=True)
c2ws = list(poses["c2ws"])
names = list(poses["names"])

server = viser.ViserServer(share=True)

for idx, (c2w, name) in enumerate(zip(c2ws, names)):
    img = cv2.imread(os.path.join("airpods", name))
    if img is None:
        continue

    H, W = img.shape[:2]
    fov = 2 * np.arctan2(H / 2.0, K[0, 0])

    server.scene.add_camera_frustum(
        f"/cameras/{idx}",
        fov=fov,
        aspect=W / float(H),
        scale=0.01,
        wxyz=viser.transforms.SO3.from_matrix(c2w[:3, :3]).wxyz,
        position=c2w[:3, 3],
        image=img[:, :, ::-1]
    )

while True:
    time.sleep(0.1)


╭────── viser (listening *:8090) ───────╮
│             ╷                         │
│   HTTP      │ http://localhost:8090   │
│   Websocket │ ws://localhost:8090     │
│             ╵                         │
╰───────────────────────────────────────╯

(viser) Share URL requested!

(viser) Generated share URL (expires in 24 hours, max 16 clients): https://action-disparity.share.viser.studio

(viser) Connection closed (0, 0 total)

(viser) Connection opened (0, 1 total), 130 persistent messages

KeyboardInterrupt: 

In [50]:
imgs = []
c2ws = []
c2ws_all = list(poses["c2ws"])
H0 = W0 = None
for idx, n in enumerate(names):
    p = os.path.join("airpods", n)
    im = cv2.imread(p)
    if im is None: 
        continue
    if im.ndim == 2:
        im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
    und = cv2.undistort(im, K, dist)
    if H0 is None:
        H0, W0 = und.shape[:2]
        focal = float(K[0,0])
    else:
        if und.shape[:2] != (H0, W0):
            und = cv2.resize(und, (W0, H0), interpolation=cv2.INTER_AREA)
    imgs.append(und)
    c2ws.append(c2ws_all[idx])

imgs = np.stack(imgs, axis=0).astype(np.uint8)
c2ws = np.array(c2ws, dtype=np.float32)

N = len(imgs)
n_train = int(0.9 * N)
n_val = int(0.10 * N)

images_train = imgs[:n_train]
c2ws_train = c2ws[:n_train]

images_val = imgs[n_train:n_train + n_val]
c2ws_val = c2ws[n_train:n_train + n_val]

images_test = imgs[n_train + n_val:]
c2ws_test = c2ws[n_train + n_val:]

np.savez(
    "calibration_output/my_nerf_data.npz",
    images_train=images_train,
    c2ws_train=c2ws_train,
    images_val=images_val,
    c2ws_val=c2ws_val,
    images_test=images_test,
    c2ws_test=c2ws_test,
    focal=focal
)